# Math 271.1: Exercise 2 (#4)

#### INSTRUCTION: Image Compression Using Singular Value Decomposition (SVD) <br>

(a) Compute the singular value decomposition A= UΣVT <br>
(b) Reconstruct the image using only the top k singular values <br>
(c) Display both the original and compressed images side by side.